In [1]:
import numpy as np
import pandas as pd
import copy
import random
random.seed(99)

In [2]:
""" 
8 hours a day: 9:00 till 5:00
total 5 days
total slots available/week: 8*5 = 40
total slots available/day: 8
total rooms availble/day: 10
total lectures possible/day: 15*40 = 400
Total teachers: 20 

Total batches: 4 (BSCS15 BSCS16 BSCS17 BSCS18)
Total sections/batch: 6 (A,B,C,D,E,F)
Total groups: 6*4 = 24


Each batch studies 5 different courses in a semester.
Total classes: 24*5 = 120
Total courses: 4*5 = 20

Every class needs  3hrs a week
Every class requires 3 slots per week

Total lectures required/week: 120*3 = 360
Total lectures required/day: 360/5 = 72

"""

' \n8 hours a day: 9:00 till 5:00\ntotal 5 days\ntotal slots available/week: 8*5 = 40\ntotal slots available/day: 8\ntotal rooms availble/day: 10\ntotal lectures possible/day: 15*40 = 400\nTotal teachers: 20 \n\nTotal batches: 4 (BSCS15 BSCS16 BSCS17 BSCS18)\nTotal sections/batch: 6 (A,B,C,D,E,F)\nTotal groups: 6*4 = 24\n\n\nEach batch studies 5 different courses in a semester.\nTotal classes: 24*5 = 120\nTotal courses: 4*5 = 20\n\nEvery class needs  3hrs a week\nEvery class requires 3 slots per week\n\nTotal lectures required/week: 120*3 = 360\nTotal lectures required/day: 360/5 = 72\n\n'

In [3]:
class Group:
    def __init__(self,name):
        self.name = name
    def getName(self):
        return self.name
    def setName(self,name):
        self.name = name
    def getCourses(self):
        return self.courses
    def setCourses(self,courses):
        self.courses = courses
    def __str__(self):
        return self.name

In [4]:
class Instructor:
    def __init__(self,i,name,courses):
        self.id = i
        self.name = name
        self.courses = courses
    def getId(self):
        return self.id
    def setId(self,i):
        self.id = i
    def getName(self):
        return self.name
    def setName(self,i):
        self.name = i
    def getCourses(self):
        return self.courses
    def setCourses(self,i):
        self.courses = i

In [5]:
class Lecture:
    def __init__(self,i,c,r,g,s):
        self.instructor = i
        self.course = c
        self.room = r
        self.group = g
        self.slot = s
    def getInstructor(self):
        return self.instructor
    def setInstructor(self,i):
        self.instructor = i
    def getCourse(self):
        return self.course
    def setCourse(self,c):
        self.course = c
    def getRoom(self):
        return self.room
    def setRoom(self,r):
        self.room = r
    def getGroup(self):
        return self.group
    def setGroup(self,g):
        self.group = g
    def __str__(self):
        return self.course.name+"("+self.group.name+")"

In [6]:
class Course:
    def __init__(self,i,name,nol):
        self.id = i
        self.name = name
        self.nol = nol    #nol means number of lectures per week
    def getId(self):
        return self.id
    def setId(self,i):
        self.id = i
    def getName(self):
        return self.name
    def setName(self,n):
        self.name = n
    def getNol(self):
        return self.nol
    def setNol(self,n):
        self.nol = n

In [7]:
class Timeslot:
    def __init__(self,slot,rooms):
        self.lectures = []
        self.slot = slot
        self.rooms = rooms
    def getLect(self):
        return self.lecture
    def setLect(self,l):
        self.lecture = l
    def getSlotNo(self):
        return self.slot_no
    def setSlotNo(self, v):
        self.slot_no = v

In [8]:
class Day:
    def __init__(self, day):
        self.day = day
        self.timeslots = []
    def setDay(self,d):
        self.id = d
    def getDay(self):
        return self.id
    def setTimeSlots(self,timeslots):
        self.timeslots = timeslots
    def getTimeSlots(self):
        return self.timeslots

In [9]:
class Timetable:
    def __init__(self,days):
        self.days = days

In [10]:
def initialize(n_b, n_s, n_i, n_r):
    n_batches = n_b
    n_sections = n_s
    n_instructors = n_i
    n_rooms = n_r
    days = ['Mon','Tue','Wed','Thu', 'Fri']
    weekdays = [Day(d) for d in days]
    
    default = "BSCS201"
    
    g = [default + str(i) + '_' + chr(j) for i in range(5,5+n_batches) for j in range(65, 65+n_sections)]
    i = ['T'+str(i) for i in range(n_instructors)]
    # Each batch has opted to study 5 unique courses
    n_courses = n_batches * 5
    c = ['C'+str(i) for i in range(n_courses)]
    courses = [Course(x,y,3) for x,y in enumerate(c)]
    groups = [Group(k) for k in g]
    d = assignCourses(courses=c, instructors=i,samplevalue=3)
    #display(d)
    idx = 0
    instructors = []
    for k,v in d.items():
        teaches = [next((x for x in courses if x.name == cname), None) for cname in v]
        instructors.append(Instructor(idx,k,teaches))
        idx+=1
    return n_courses,n_rooms,n_instructors,n_sections,weekdays,courses,groups,instructors


In [116]:
def generate_timetable(groups, courses, instructors, weekdays, n_rooms):
    timeslots = []
    slots = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-1:00', '1:00-2:00','2:00-3:00','3:00-4:00','4:00-5:00']
    for s in slots:
        rooms = [i for i in range(0,n_rooms)]
        timeslots.append(Timeslot(s,rooms))
    # print(timeslots[0].slot, timeslots[0].room)
    for d in weekdays:
        d.timeslots = timeslots.copy()
    
    for day in weekdays:
        np.random.shuffle(day.timeslots)
        for slot in day.timeslots:
            lectures = []
            groups_ = groups.copy()
            m=0
            np.random.shuffle(slot.rooms)
            for room in slot.rooms:
                if len(groups_)==0:
                    break
                group_ = random.choice(groups_)
                # groups_.remove(group_)
                course = random.choice(courses)
                i = random.choice(instructors)
                while course not in i.courses:
                    i = random.choice(instructors)
                lectures.append (Lecture(i,course,room,group_,slot.slot))
                m+=1
            slot.lectures = copy.deepcopy(lectures)
    t = Timetable(weekdays)
    #print(t.days[1].timeslots[2].lectures[10])
    
    return copy.deepcopy(t)


        




In [117]:
def assignCourses(courses,instructors,samplevalue):
    d = {}
    old = []
    c = []
    for i in instructors:
        while c == old:  
            c = random.sample(courses, k = samplevalue)
        d[i] = c
        old = c
    tmp =[i for k,v in d.items() for i in v]
    not_assigned = [i for i in tmp + courses if i not in tmp or i not in courses]
    while len(not_assigned)!=0:
        e = not_assigned.pop()
        d[random.choice(instructors)].append(e)
    return copy.deepcopy(d)
def decimalToBinary(n):  
    return bin(n)

In [114]:
def display_timetable(timetable):
    from prettytable import PrettyTable
    column_names = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-1:00', '1:00-2:00','2:00-3:00','3:00-4:00','4:00-5:00']
    for day in timetable.days:
        table = PrettyTable()
        table.add_column("Room",day.timeslots[0].rooms)
        print(day.day)
        
        for ts in day.timeslots:
            display = []
            for r in ts.rooms:
                boolean = False
                for lect in ts.lectures:
                    if lect.room == r:
                        display.append(lect)
                        boolean = True
                if not boolean:
                    display.append("")
        
        for c in column_names:
            table.add_column(c,display)

        print(table)

In [135]:
class GeneticAlgorithm:
    def __init__(self, mutation, population_size, n_offspring, n_gen, summary):
        self.mutation = mutation
        self.summary = summary
        self.population_size = population_size
        self.n_offspring = n_offspring
        self.n_gen = n_gen
    
    def producePopulation(self, n_b, n_s, n_i, n_r):
        n_courses,n_rooms,n_instructors,n_sections,weekdays,courses,groups,instructors = initialize(n_b,n_s,n_i,n_r)
        self.timetables = []
        i = 0
        while i < self.population_size:
            t = generate_timetable(groups, courses, instructors, weekdays, n_rooms)
            self.timetables.append(t)
            i+=1

    def fitnessFunction(self):
        self.costs = []
        self.chromosomes = []
        for t in self.timetables:
            cost, clashincidents,chromosome = self.fitness(t)
            self.costs.append(cost)
            self.chromosomes.append(chromosome)
        
        return self.costs ,self.chromosomes

    def selection(self,costs,chromosomes):
        S = np.sum(costs)
        pick = random.uniform(0, S)
        current = 0
        parents = []
        for i in range(2):
            for i,c in enumerate(chromosomes):
                current+=costs[i]
                if current > pick:
                    parents.append(c)
        return parents
    
    def mutate(self,offsprings):
        for os in offsprings:
            for gene in os:
                gene_ = list(gene)
                for n in range(self.mutation):
                    pick = random.randint(0,len(gene_)-1)
                    if gene_[pick] == '0':
                        gene_[pick] = '1'
                    else:
                        gene_[pick] = '0'
                    gene = ''.join(gene_).strip()




    def crossover(self):
        summary = self.summary
        day_bits = 3
        slot_bits = 3
        course_bits = np.floor(np.log2(summary['n_courses'])+1)
        instructor_bits = np.floor(np.log2(summary['n_instructors'])+1)
        batch_bits = np.floor(np.log2(summary['n_batches']-1)+1)
        sec_bits = np.floor(np.log2(summary['n_sections']-1)+1)
        room_bits = np.floor(np.log2(summary['n_rooms'])+1)
        order = {
            'day': day_bits,
            'slot': slot_bits,
            'course': course_bits,
            'instructor': instructor_bits,
            'batch': batch_bits,
            'sec': sec_bits,
            'room': room_bits,
        }

        costs,chromosomes = self.fitnessFunction()
        for x in range(self.n_gen):
            parents = self.selection(self.costs, self.chromosomes)

            # 2 parents
            p1 = parents[0]
            p2 = parents[1]
            offsprings = []
            for n in range(self.n_offspring):
                chromosome = []
                for g1,g2 in zip(p1,p2):
                    point = random.randint(0,len(g1))
                    point2 = random.randint(0,len(g2))
                    slice1 = g1[:point]
                    slice2 = g2[point:point2]
                    slice3 = g1[point2:]
                    gene1 = slice1+slice2+slice3

                    # print(gene)
                    '''
                    for k,v in order.items():
                        v = int(v)
                        for idx in range(v):
                            fate = random.uniform(0,10)
                            if fate < 5:
                                #print('bit choseng1: ', g1[idx])
                                gene.append(g1[idx])
                            else:
                                #print('bit choseng2: ', g2[idx])
                                gene.append( g2[idx])
                    gene = ''.join(gene)
                    gene = gene.strip()
                    '''
                    # print(order,offspring,sep='\n')
                    if self.validGene(gene1):
                        chromosome.append(gene1)
                offsprings.append(chromosome)

            self.mutate(offsprings)  
            print('Gen: ',x)
            for os in offsprings:
                # print(os[0])
                #t = self.getTimetable(os)
                cost, clashincidents,chromosome = self._fitness(os)
                idx = np.argmin(self.costs)
                print('offspring'+str(x)+' fitness:'  , cost)
                print('current min fitness:',self.costs[idx])
                if cost > self.costs[idx]:
                    self.costs[idx] = cost
                    #print("---",os,sep='\n')
                    self.chromosomes[idx] = os

        idx = np.argmax(self.costs)
        return self.chromosomes[idx], self.costs[idx] 

    def execute(self):
        chromosome, cost = self.crossover()
        return chromosome,cost

    def getSchedule(self,chromosome):
        abstracts = [self.decode(c) for c in chromosome]
        df = pd.DataFrame(abstracts)
        days = ['Mon','Tue','Wed','Thu', 'Fri']
        slots = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-1:00', '1:00-2:00','2:00-3:00','3:00-4:00','4:00-5:00']
        for d in df.day.unique():
            df.loc[df['day'] == d, 'day'] = days[d]
        for s in df.slot.unique():
            df.loc[df['slot'] == s, 'slot'] = slots[s]
        return df

    def getTimetable(self,chromosome):
        abstracts = [self.decode(c) for c in chromosome]
        df = pd.DataFrame(abstracts)
        #return df[df.day == 0][df.slot == 0]
        # day	slot	course	instructor	batch	sec	room

        #days
        days_id = np.sort(df.day.unique())
        days = ['Mon','Tue','Wed','Thu', 'Fri']
        weekdays = [Day(days[i]) for i in days_id]

        #rooms
        rooms = np.sort(df.room.unique())

        #slots
        slots_id = np.sort(df.slot.unique())
        slots = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-1:00', '1:00-2:00','2:00-3:00','3:00-4:00','4:00-5:00']
        timeslots = [Timeslot(slots[i],rooms) for i in slots_id]
        for wd in weekdays:
            wd.timeslots = timeslots

        #Courses
        courses_id = np.sort(df.course.unique())
        courses = [Course(i,'C'+str(i),3) for i in courses_id]

        #instructors
        instructors_id = np.sort(df.instructor.unique())
        instructors = []
        for i in instructors_id:
            df1 = df[df.instructor==i]
            i_courses = df1.course.unique()
            instructors.append(Instructor(i,'T'+str(i),i_courses))

        #groups
        batches = df.batch.unique()
        groups = []
        for b in batches:
            df1 = df[df.batch == b]
            b_sec = df1.sec.unique()
            for s in b_sec:
                groups.append(Group(b+'_'+s))
        

        #for index, row in df.iterrows():
        for _d in days_id:
            df_1 = df[df.day == _d]
            day = _d
            day = days[day]
            # print(day)
            for wd in weekdays:
                if wd.day == day:
                    for _s in df_1.slot.unique():
                        df_2 = df_1[df_1.slot == _s]
                        # print("Same day, same slot: n_lectures: ",len(df_2))
                        slot = _s
                        slot = slots[slot]
                        # print(slot)
                        for s in wd.timeslots:
                            if s.slot == slot:
                                for index, row in df_2.iterrows():
                            
                                    inst = 'T'+str(row.instructor)
                                    for i in instructors:
                                        if i.name == inst:
                                            inst = i
                                            break
                                    course = 'C'+str(row.course)
                                    for c in courses:
                                        if c.name == course:
                                            course = c
                                            break
                                    batch = row.batch
                                    sec = row.sec
                                    group = batch+'_'+sec
                                    for g in groups:
                                        if g.name == group:
                                            group = g
                            
                                    room = row.room
                                    
                                    # group slot day inst course room
                                    lect = Lecture(inst,course,room,group,s.slot)
                                    #print(lect)
                                    s.lectures.append(lect)
                            
                break
                # day.timeslots.append(slot)
        tt = Timetable(weekdays)
        return copy.deepcopy(tt)


            
    def validGene(self,gene):
        offspring = gene
        summary = self.summary
        day_bits = 3
        slot_bits = 3
        course_bits = np.floor(np.log2(summary['n_courses'])+1)
        instructor_bits = np.floor(np.log2(summary['n_instructors'])+1)
        batch_bits = np.floor(np.log2(summary['n_batches']-1)+1)
        sec_bits = np.floor(np.log2(summary['n_sections']-1)+1)
        room_bits = np.floor(np.log2(summary['n_rooms'])+1)
        order = {
            'day': day_bits,
            'slot': slot_bits,
            'course': course_bits,
            'instructor': instructor_bits,
            'batch': batch_bits,
            'sec': sec_bits,
            'room': room_bits,
        }
        sample = copy.deepcopy(offspring)
        for k,v in order.items():
            v = int(v)
            test = int(sample[:v],2)
            if k == 'day' and test>=5:
                return False
            elif k=='slot' and test>=8:
                return False
            elif k=='course' and test>=summary['n_courses']:
                return False
            elif k=='batch' and test>=summary['n_batches']:
                return False
            elif k=='sec' and test>=summary['n_sections']:
                return False
            elif k=='room' and test>=summary['n_rooms']:
                return False
            sample = sample[v:]
        return True    

    def createGene(self, lect, day):
        # Day, Slot,course,instructor,batch,section,room
        summary = self.summary

        # Day
        day = day.day
        mapping = {
            'Mon':0,
            'Tue':1,
            'Wed':2,
            'Thu':3,
            'Fri':4
        }
        day_id = mapping[day]
        bits = int(np.floor(np.log2(4)+1))
        _bin = bin(day_id)[2:].zfill(bits)

        mapping = {
            '9:00-10:00': 0, 
            '10:00-11:00':1, 
            '11:00-12:00':2,
             '12:00-1:00':3,
              '1:00-2:00':4,
              '2:00-3:00':5,
              '3:00-4:00':6,
              '4:00-5:00':7,
            }
        slot_id = mapping[lect.slot]
        # 3 bits to represent slot number
        bits = np.floor(np.log2(7)+1)
        bits = int(bits)
        _bin += bin(slot_id)[2:].zfill(bits)

        # courses
        bits = np.floor(np.log2(summary['n_courses'])+1)
        bits = int(bits)
        
        _bin+=bin(int(lect.course.name.replace('C','')))[2:].zfill(bits)

        # instructors
        bits = np.floor(np.log2(summary['n_instructors'])+1)
        bits = int(bits)
        _bin+=bin(int(lect.instructor.name.replace('T','')))[2:].zfill(bits)

        #batch
        mapping = {}
        for i in range(summary['n_batches']):
            mapping[chr(53+i)] = i
        tmp = lect.group.name.replace('BSCS201','')
        tmp = tmp.replace('_','')
        tmp = tmp[:len(tmp)-1]
        batch = mapping[tmp]
        bits = np.floor(np.log2(summary['n_batches']-1)+1)
        bits = int(bits)
        _bin+=bin(batch)[2:].zfill(bits)

        #section
        mapping = {}
        for i in range(summary['n_sections']):
            mapping[chr(65+i)] = i
        tmp = lect.group.name.replace('BSCS201','')
        tmp = tmp.replace('_','')
        tmp = tmp[len(tmp)-1]
        sec = mapping[tmp]
        bits = np.floor(np.log2(summary['n_sections']-1)+1)
        bits = int(bits)
        _bin+=bin(sec)[2:].zfill(bits)

        #room
        bits = np.floor(np.log2(summary['n_rooms'])+1)
        bits = int(bits)
        _bin+=bin(lect.room)[2:].zfill(bits)
        return _bin
    

    def decode(self,chromosome):
        summary = self.summary
        day_bits = 3
        slot_bits = 3
        course_bits = np.floor(np.log2(summary['n_courses'])+1)
        instructor_bits = np.floor(np.log2(summary['n_instructors'])+1)
        batch_bits = np.floor(np.log2(summary['n_batches']-1)+1)
        sec_bits = np.floor(np.log2(summary['n_sections']-1)+1)
        room_bits = np.floor(np.log2(summary['n_rooms'])+1)
        order = {
            'day': day_bits,
            'slot': slot_bits,
            'course': course_bits,
            'instructor': instructor_bits,
            'batch': batch_bits,
            'sec': sec_bits,
            'room': room_bits,
        }
        for k,v in order.items():
            v = int(v)
            if k == 'batch':
                tmp = int(chromosome[:v],2)
                mapping = {}
                for i in range(summary['n_batches']):
                    mapping[i] = chr(53+i)
                batch = mapping[tmp]
                order[k] = "BSCS201"+batch
            elif k == 'sec':
                tmp = int(chromosome[:v],2)
                mapping = {}
                for i in range(summary['n_sections']):
                    mapping[i] = chr(65+i)
                sec = mapping[tmp]
                order[k] = sec
            else:
                order[k] = int(chromosome[:v],2)
            chromosome = chromosome[v:]
        return copy.deepcopy(order)

    def createChromosome(self,t):
        chromosome = []
        for day in t.days:
            for slot in day.timeslots:
                for lect in slot.lectures:
                    chromosome.append(self.createGene(lect,day))
        
        return copy.deepcopy(chromosome)

    def fitness(self,t):
        chromosome = self.createChromosome(t)
        abstracts = [self.decode(c) for c in chromosome]
        df = pd.DataFrame(abstracts)
        # display(df)
        # Same time
        days = df.day.unique()
        slots = df.slot.unique()
        cost = 15000
        # A teacher cannot teach more than one classes at a time.
        teacher_clash = 0
        for d in days:
            df1 = df[df.day == d]
            for s in slots:
                df2 = df1[df1.slot == s]
                instructors = df2.instructor.unique()
                for i in instructors:
                    df3 = df2[df2.instructor == i]
                    if len(df3)>=2:
                        teacher_clash+=1
                        cost-=10
        # print(teacher_clash)

        # No section can listen for two classes at the same time
        section_clash = 0
        for d in days:
            df1 = df[df.day == d]
            for s in slots:
                df2 = df1[df1.slot == s]
                batches = df2.batch.unique()
                for b in batches:
                    df3 = df2[df2.batch == b]
                    sects = df3.sec.unique()
                    for s in sects:
                        df4 = df3[df3.sec == s]
                        if len(df4)>=2:
                            section_clash+=1
                            cost-=10
        # print(section_clash)
        
        # No classroom can receive two classes at the same time
        room_clash = 0
        for d in days:
            df1 = df[df.day == d]
            for s in slots:
                df2 = df1[df1.slot == s]
                rooms = df2.room.unique()
                for r in rooms:
                    df3 = df2[df2.room == r]
                    if len(df3)>=2:
                        room_clash+=1
                        cost-=10
        # print(room_clash)

        # No teacher can hold three consecutive classes
        hectic_clash = 0
        for d in days:
            df1 = df[df.day == d]
            instructors = df1.instructor.unique()
            for i in instructors:
                df2 = df1[df1.instructor == i]
                for s in df2.slot:
                    if len(df2[df2.slot == s])>=2:
                        hectic_clash+=1

                    if len(df2[df2.slot == s+1])>=1:
                        if len(df2[df2.slot == s+2])>=1:
                            hectic_clash+=1
                            cost-=5
        # print(hectic_clash)
        clashincidents = {
            "A teacher cannot teach more than one classes at a time" : teacher_clash,
            "No section can listen for two classes at the same time": section_clash,
            "No classroom can receive two classes at the same time": room_clash,
            "No teacher can hold three consecutive classes": hectic_clash,
        }            
        return cost, copy.deepcopy(clashincidents),chromosome

    def _fitness(self,chromosome):
        abstracts = [self.decode(c) for c in chromosome]
        df = pd.DataFrame(abstracts)
        #print('unique instructors:', df.instructor.unique())
        #print('unique batches:', df.batch.unique())
        #print('unique sects:', df.sec.unique())
        #print('unique rooms:', df.room.unique())
        # display(df.head(4))
        # display(df)
        # Same time
        days = df.day.unique()
        #print('unique days:', days)
        slots = df.slot.unique()
        #print('unique slots:', slots)
        cost = 15000
        # A teacher cannot teach more than one classes at a time.
        teacher_clash = 0
        for d in days:
            df1 = df[df.day == d]
            for s in slots:
                df2 = df1[df1.slot == s]
                instructors = df2.instructor.unique()
                for i in instructors:
                    df3 = df2[df2.instructor == i]
                    if len(df3)>=2:
                        teacher_clash+=1
                        cost-=10
        # print(teacher_clash)

        # No section can listen for two classes at the same time
        section_clash = 0
        for d in days:
            df1 = df[df.day == d]
            for s in slots:
                df2 = df1[df1.slot == s]
                batches = df2.batch.unique()
                for b in batches:
                    df3 = df2[df2.batch == b]
                    sects = df3.sec.unique()
                    for s in sects:
                        df4 = df3[df3.sec == s]
                        if len(df4)>=2:
                            section_clash+=1
                            cost-=10
        # print(section_clash)
        
        # No classroom can receive two classes at the same time
        room_clash = 0
        for d in days:
            df1 = df[df.day == d]
            for s in slots:
                df2 = df1[df1.slot == s]
                rooms = df2.room.unique()
                for r in rooms:
                    df3 = df2[df2.room == r]
                    if len(df3)>=2:
                        room_clash+=1
                        cost-=10
        # print(room_clash)

        # No teacher can hold three consecutive classes
        hectic_clash = 0
        for d in days:
            df1 = df[df.day == d]
            instructors = df1.instructor.unique()
            for i in instructors:
                df2 = df1[df1.instructor == i]
                for s in df2.slot:
                    if len(df2[df2.slot == s])>=2:
                        hectic_clash+=1

                    if len(df2[df2.slot == s+1])>=1:
                        if len(df2[df2.slot == s+2])>=1:
                            hectic_clash+=1
                            cost-=5
        # print(hectic_clash)
        clashincidents = {
            "A teacher cannot teach more than one classes at a time" : teacher_clash,
            "No section can listen for two classes at the same time": section_clash,
            "No classroom can receive two classes at the same time": room_clash,
            "No teacher can hold three consecutive classes": hectic_clash,
        }            
        return cost, copy.deepcopy(clashincidents),chromosome




In [146]:

summary = {
    'n_batches': 4,
    'n_courses': 4*5,
    'n_rooms': 25,
    'n_instructors': 20,
    'n_sections': 6
}
mutation = 0
population = 10
offsprings = 1
generations = 10
GA = GeneticAlgorithm(0,10,1,50,summary)

GA.producePopulation(4,6,20,25)
chromosome,fitness = GA.execute()
print(fitness)
GA.getSchedule(chromosome)


Gen:  0
offspring0 fitness: 6930
current min fitness: 7050
Gen:  1
offspring1 fitness: 7300
current min fitness: 7050
Gen:  2
offspring2 fitness: 6775
current min fitness: 7050
Gen:  3
offspring3 fitness: 6780
current min fitness: 7050
Gen:  4
offspring4 fitness: 6690
current min fitness: 7050
Gen:  5
offspring5 fitness: 6740
current min fitness: 7050
Gen:  6
offspring6 fitness: 8195
current min fitness: 7050
Gen:  7
offspring7 fitness: 6870
current min fitness: 7075
Gen:  8
offspring8 fitness: 7195
current min fitness: 7075
Gen:  9
offspring9 fitness: 6325
current min fitness: 7195
Gen:  10
offspring10 fitness: 8255
current min fitness: 7195
Gen:  11
offspring11 fitness: 6720
current min fitness: 7300
Gen:  12
offspring12 fitness: 6675
current min fitness: 7300
Gen:  13
offspring13 fitness: 8890
current min fitness: 7300
Gen:  14
offspring14 fitness: 9830
current min fitness: 7300
Gen:  15
offspring15 fitness: 9145
current min fitness: 7375
Gen:  16
offspring16 fitness: 9035
current m

,day,slot,course,instructor,batch,sec,room
0,Mon,2:00-3:00,8,15,BSCS2015,F,10
1,Mon,1:00-2:00,19,18,BSCS2017,A,1
2,Mon,2:00-3:00,1,6,BSCS2015,F,2
3,Mon,2:00-3:00,16,7,BSCS2017,D,18
4,Mon,2:00-3:00,12,17,BSCS2016,B,4
...,...,...,...,...,...,...,...
297,Wed,10:00-11:00,19,9,BSCS2017,A,8
298,Wed,9:00-10:00,15,10,BSCS2018,F,10
299,Wed,2:00-3:00,2,5,BSCS2018,A,18
300,Wed,10:00-11:00,19,8,BSCS2017,D,13
